In [1]:
from PIL import Image

import numpy as np
import pandas as pd

import torch
import torchvision.models as models
import torchvision.transforms as transforms


In [2]:

# Pre Processing

df = pd.read_csv('/kaggle/input/breakhis/Folds.csv')
df["class"] = df["filename"].str.split('/').str[3]
df["filename"] = '/kaggle/input/breakhis/BreaKHis_v1/' + df["filename"]

In [3]:
from torchvision.models import resnet50
from torchvision.models import ResNet50_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Carregar o ResNet-50
resnet50 = models.resnet50(weights=ResNet50_Weights.DEFAULT).to(device)
resnet50.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
resnet50 = torch.nn.Sequential(*(list(resnet50.children())[:-1]))
resnet50.eval()


def extract_features_resnet50(filename):
    img = Image.open(filename).convert('RGB')
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    img_tensor = transform(img).to(device)
    img_tensor = img_tensor.unsqueeze(0)
    
    with torch.no_grad():
        features = resnet50(img_tensor)
        #features = resnet50_avgpool(features)
        features = torch.flatten(features)
        
    return features.cpu().numpy()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 144MB/s]


In [4]:
X_train_resnet50, y_train_resnet50, X_test_resnet50, y_test_resnet50 = list(), list(), list(), list()

base = df.query("fold == 1").query("mag == 400")

for index, row in base.iterrows():    
    features = extract_features_resnet50(row["filename"])
    if row['grp'] == 'train':
        X_train_resnet50.append(features)
        y_train_resnet50.append(row["class"])
    else:
        X_test_resnet50.append(features)
        y_test_resnet50.append(row["class"])

# Convert to numpy arrays
X_train_resnet50, y_train_resnet50, X_test_resnet50, y_test_resnet50 = np.array(X_train_resnet50), np.array(y_train_resnet50), np.array(X_test_resnet50), np.array(y_test_resnet50)

print('Treino:', X_train_resnet50.shape, y_train_resnet50.shape)
print('Teste:', X_test_resnet50.shape, y_test_resnet50.shape)

Treino: (1165, 2048) (1165,)
Teste: (655, 2048) (655,)


In [5]:
np.save('./resnet50_X_train.npy', X_train_resnet50)
np.save('./resnet50_y_train.npy', y_train_resnet50)
np.save('./resnet50_X_test.npy', X_test_resnet50)
np.save('./resnet50_y_test.npy', y_test_resnet50)

In [6]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Flatten the array along the channel dimension
#X_train_resnet50_flat = X_train_resnet50.reshape(X_train_resnet50.shape[0]*X_train_resnet50.shape[1], X_train_resnet50.shape[2])
#X_test_resnet50_flat = X_test_resnet50.reshape(X_test_resnet50.shape[0]*X_test_resnet50.shape[1], X_test_resnet50.shape[2])

# Normalizando
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train_resnet50)
X_test_scaled = ss.transform(X_test_resnet50)


# Testando com SVM
clf = SVC()
clf.fit(X_train_scaled, y_train_resnet50)
y_pred_resnet50 = clf.predict(X_test_scaled)

print(classification_report(y_test_resnet50, y_pred_resnet50))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


              precision    recall  f1-score   support

      benign       0.88      0.54      0.67       237
   malignant       0.78      0.96      0.86       418

    accuracy                           0.81       655
   macro avg       0.83      0.75      0.76       655
weighted avg       0.82      0.81      0.79       655

